In [44]:
import qiskit
from qiskit import IBMQ, QuantumCircuit, assemble
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.test.mock import FakeAlmaden, FakeArmonk, FakeAthens
from qiskit.tools import job_monitor
from qutip import sigmax, sigmay, Qobj, sigmaz
from src.QutipOptimizer import QutipOptimizer
from src.prefactor_parsing import prefactor_parser
import numpy as np

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
backend = FakeArmonk()

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_backend = provider.get_backend('ibmq_armonk')
hamiltonian_backend = real_backend.configuration().hamiltonian

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ibmqfactory.load_account:WARNING:2020-08-20 11:29:06,898: Credentials are already in use. The existing account in the session will be replaced.


In [81]:
num_qub=1

circ = QuantumCircuit(num_qub)

circ.x(0)
# circ.x(2)
circ.measure_all()


/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [97]:
lst = ['omegad' + str(i) for i in range(num_qub)]
for ele in lst:
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 0.5 * getattr(real_backend.configuration(), 'hamiltonian')['vars'][ele]
# freq_est = 4.97e9
# armonk_backend.defaults().qubit_freq_est = [freq_est]
lst2 = ['wq' + str(i) for i in range(num_qub)]
for i,ele in enumerate(lst2):
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 1e-11 * 2 * np.pi * real_backend.defaults().qubit_freq_est[i]
# backend.defaults().qubit_freq_est = real_backend.defaults().qubit_freq_est

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
freq_ests = [a for a in real_backend.defaults().qubit_freq_est]
freq_ests

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[4974441106.581355]

In [99]:
grape_optimizer = QutipOptimizer(backend, n_ts=800)
builtin_instructions = backend.defaults().instruction_schedule_map

qoc_inst_map = QOCInstructionScheduleMap.from_inst_map(grape_optimizer, builtin_instructions)
grape_schedule = qiskit.schedule(circ, inst_map = qoc_inst_map,
                        meas_map = real_backend.configuration().meas_map)

grape_qobj = assemble(grape_schedule,
                      backend=real_backend,
                      qubit_lo_freq=freq_ests,
                      meas_level=2,
                      meas_return='single',
                      shots=4096)

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:qutip.control.dynamics:Setting memory optimisations for level 0
INFO:qutip.control.dynamics:Internal operator data type choosen to be <class 'numpy.ndarray'>
INFO:qutip.control.dynamics:phased dynamics generator caching True
INFO:qutip.control.dynamics:propagator gradient caching True
INFO:qutip.control.dynamics:eigenvector adjoint caching True
INFO:qutip.control.dynamics:use sparse eigen decomp False
INFO:qutip.control.pulseoptim:System configuration:
Drift Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.15627668  0.        ]
 [ 0.         -0.1562

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[-1.88525892e-07-1.42480837e-07j -2.92188893e-07-1.00000000e+00j]
 [ 2.92188895e-07-1.00000000e+00j -1.88525892e-07+1.42480835e-07j]]
1
{'D0': [(-0.5437458062305677-0.5103649681246546j), (-0.4115170444289277-0.20606285504979796j), (-0.5475366458810328-0.34385081806102613j), (-0.03197150658934064-0.36621477377186246j), (-0.44171520190397434-0.22559913377103802j), (-0.6515039052363845-0.23600075522016636j), (-0.09137567686023207-0.31520068967071463j), (-0.19311821137855534-0.4705369557405152j), (-0.33371279888152305-0.5539507667017489j), (-0.2259396279793167-0.4826045102170483j), (-0.08523694146824333-0.7j), (-0.42533118507956613-0.0940906394487847j), (-0.014235907946743275-0.3559405183675899j), (-0.23149902894806565-0.1430872004308467j), (-0.534433156238791-0.05843360037559854j), (-0.4389759565526822-0.4407925013985808j), (0.08142589502009602-0.12901333061711026j), (-0.4675726152014317-0.62726817

In [100]:
job = real_backend.run(grape_qobj)
job_monitor(job)

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Job Status: job has successfully run


In [ ]:
lst = ['omegad' + str(i) for i in range(num_qub)]
for ele in lst:
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 0.5 * 1e9 * getattr(real_backend.configuration(), 'hamiltonian')['vars'][ele]
# freq_est = 4.97e9
# armonk_backend.defaults().qubit_freq_est = [freq_est]
lst2 = ['wq' + str(i) for i in range(num_qub)]
for i,ele in enumerate(lst2):
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 2 * np.pi * real_backend.defaults().qubit_freq_est[i]
# backend.defaults().qubit_freq_est = real_backend.defaults().qubit_freq_est

In [ ]:
backend_sim = PulseSimulator()
pulse_model = PulseSystemModel.from_backend(backend)

In [ ]:
def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
default_schedule = qiskit.schedule(def_trans_circ, inst_map = backend.defaults().instruction_schedule_map,
                        meas_map = real_backend.configuration().meas_map)


default_qobj = assemble(default_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=freq_ests,
                      meas_level=2,
                      meas_return='single',
                      shots=1024)


In [ ]:
sim_result = backend_sim.run(grape_qobj, pulse_model).result()

In [ ]:
sim_result.get_counts()

In [ ]:
job = real_backend.run(grape_qobj)

In [ ]:
job_monitor(job)